# P-MEDM Toy Example

This illustration replicates an example used in Tuccillo and Spielman (2022).

* **Joseph V. Tuccillo & Seth E. Spielman** (2022) A Method for Measuring Coupled Individual and Social Vulnerability to Environmental Hazards, *Annals of the American Association of Geographers*, 112:6, 1702-1725, DOI: [10.1080/24694452.2021.1989283](https://www.tandfonline.com/doi/full/10.1080/24694452.2021.1989283)

## Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2025-03-20T16:12:20.740418-04:00

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 9.0.2

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib

import numpy as np
import pandas as pd

import pmedm_legacy

In [3]:
%watermark -w
%watermark -iv

Watermark: 2.5.0

pandas      : 2.2.3
pmedm_legacy: 0.0.5
numpy       : 2.2.4



## Inputs
1. Year (`year`)
2. Microdata sample weights (`wt`)
3. Individual-Level constraints from microdata (`cind`)
4. Geographic constraints at target scale (census block groups) (`cg2` / `cbg`) and parent scale (census tracts) (`cg1` / `ctrt`)
5. Geographic constraint standard errors at target scale (census block groups) (`sg2` / `sbg`) and parent scale (census tracts) (`sg1` / `strt`)

## Load the individual constraints

In [4]:
ddir = pathlib.Path("..", "pmedm_legacy", "data")
dpattern = "toy_constraints_{}.csv"

In [5]:
cind = pd.read_csv(ddir / dpattern.format("ind"))
cind.head()

,SERIAL,PERWT,POP,CONST1,CONST2,CONST3,CONST4,CONST5,CONST6
0,A,50,1,1,1,1,0,1,0
1,B,20,1,0,0,0,0,1,1
2,C,10,1,1,1,0,1,1,1
3,D,5,1,1,0,0,0,0,0
4,E,15,1,0,1,1,0,0,0


## Sample weights

In [6]:
wt = cind["PERWT"].to_numpy()
wt

array([50, 20, 10,  5, 15])

In [7]:
cind = cind.drop(columns=["SERIAL", "PERWT"])
cind

,POP,CONST1,CONST2,CONST3,CONST4,CONST5,CONST6
0,1,1,1,1,0,1,0
1,1,0,0,0,0,1,1
2,1,1,1,0,1,1,1
3,1,1,0,0,0,0,0
4,1,0,1,1,0,0,0


# Geographic constraints

In [8]:
cbg0 = pd.read_csv(ddir / dpattern.format("bg"))
cbg0.index = cbg0["GEOID"].to_numpy()
cbg0

,GEOID,POP,POPs,CONST1,CONST1s,CONST2,CONST2s,CONST3,CONST3s,CONST4,CONST4s,CONST5,CONST5s,CONST6,CONST6s
10,10,28,2,15,1.75,25,2.25,15,3,3,0.5,16,2.75,5,1
11,11,20,2,13,1.50,13,2.25,12,2,2,0.5,17,2.75,8,1
20,20,30,2,18,1.75,22,2.25,20,2,3,0.5,27,2.75,7,1
21,21,22,2,19,1.75,15,2.25,18,2,2,0.5,20,2.75,10,1


## Separate ests and standard errors

In [9]:
se_cols = cbg0.columns[np.where([k.endswith("s") for k in cbg0.columns])]
est_cols = cbg0.columns[
    np.where([(k not in se_cols) & (k != "GEOID") for k in cbg0.columns])
]

cbg = cbg0[est_cols].copy()
sbg = cbg0[se_cols].copy()

## Tract constraints

In [10]:
ctrt0 = cbg0.copy()
ctrt0.index = [str(i)[0] for i in ctrt0["GEOID"].to_numpy()]
ctrt0 = ctrt0.drop(columns=["GEOID"])

## Separate ests and standard errors

In [11]:
ctrt = ctrt0[est_cols].copy()
strt = ctrt0[se_cols].copy()

In [12]:
ctrt

,POP,CONST1,CONST2,CONST3,CONST4,CONST5,CONST6
1,28,15,25,15,3,16,5
1,20,13,13,12,2,17,8
2,30,18,22,20,3,27,7
2,22,19,15,18,2,20,10


In [13]:
strt

,POPs,CONST1s,CONST2s,CONST3s,CONST4s,CONST5s,CONST6s
1,2,1.75,2.25,3,0.5,2.75,1
1,2,1.50,2.25,2,0.5,2.75,1
2,2,1.75,2.25,2,0.5,2.75,1
2,2,1.75,2.25,2,0.5,2.75,1


## Aggregate ests

In [14]:
ctrt = ctrt.groupby(ctrt.index).aggregate("sum")
ctrt

,POP,CONST1,CONST2,CONST3,CONST4,CONST5,CONST6
1,48,28,38,27,5,33,13
2,52,37,37,38,5,47,17


## Aggregate SE's

In [15]:
strt = strt.groupby(strt.index).aggregate(lambda x: np.sqrt(np.sum(np.square(x))))
strt

,POPs,CONST1s,CONST2s,CONST3s,CONST4s,CONST5s,CONST6s
1,2.828427,2.304886,3.181981,3.605551,0.707107,3.889087,1.414214
2,2.828427,2.474874,3.181981,2.828427,0.707107,3.889087,1.414214


## Create P-MEDM object & solve

In [16]:
pmd = pmedm_legacy.PMEDM(
    2019,
    wt,
    cind,
    cbg,
    ctrt,
    sbg,
    strt,
)
pmd.solve()

Using Sparse Hessian Preconditioner
Initializing PMEDM solver
Starting Trust Region Algorithm
 iter          fun     ||grad|| ||try step||      act red     pred red     radius inner iter
    0            0     0.229728            1      1.53947      1.54141          1           0
    1     -1.53947     0.125299      1.04989      0.54656     0.542251          2          12
    2     -2.08603    0.0237954      0.20596   0.00612304   0.00502324          4          15
    3     -2.09216   0.00706137     0.146657   0.00124603   0.00109171          8          16
    4      -2.0934   0.00142816     0.045663  7.04901e-05   6.8018e-05         16          16
    5     -2.09347  7.67754e-05   0.00272249  2.17343e-07  2.16905e-07         32          16
    6     -2.09347  2.32307e-07  8.27217e-06  1.98996e-12  1.99051e-12         64          16
    7     -2.09347  2.12434e-12           -            -            -          -           - 
Solver ended with convergence


## Inspect results

In [17]:
pmd.pmedm_res

{'almat': array([[ 9.88971079,  9.48999564, 16.21106451, 14.41895525],
        [ 2.261975  ,  5.85162672,  4.97978835,  6.90333539],
        [ 3.07611053,  2.01920552,  2.83472006,  2.0695863 ],
        [ 2.53937691,  1.32763304,  0.34660813,  0.76989277],
        [ 9.40498296,  1.56332278,  3.94898796,  0.09312139]]),
 'serialno': array(['0', '1', '2', '3', '4'], dtype='<U1'),
 'geoid': array(['10', '11', '20', '21'], dtype='<U2')}

## Perform diagnostics

In [18]:
pmedm_legacy.diagnostics.moe_fit_rate(cind, cbg, sbg, pmd.pmedm_res["almat"])

{'Ycomp':    variable  acs      pmedm       err      moe  in_moe
 10      POP   28  27.172156  0.827844  3.29000    True
 11      POP   20  20.251784  0.251784  3.29000    True
 20      POP   30  28.321169  1.678831  3.29000    True
 21      POP   22  24.254891  2.254891  3.29000    True
 10   CONST1   15  15.505198  0.505198  2.87875    True
 11   CONST1   13  12.836834  0.163166  2.46750    True
 20   CONST1   18  19.392393  1.392393  2.87875    True
 21   CONST1   19  17.258434  1.741566  2.87875    True
 10   CONST2   25  22.370804  2.629196  3.70125    True
 11   CONST2   13  13.072524  0.072524  3.70125    True
 20   CONST2   22  22.994773  0.994773  3.70125    True
 21   CONST2   15  16.581663  1.581663  3.70125    True
 10   CONST3   15  19.294694  4.294694  4.93500    True
 11   CONST3   12  11.053318  0.946682  3.29000    True
 20   CONST3   20  20.160052  0.160052  3.29000    True
 21   CONST3   18  14.512077  3.487923  3.29000   False
 10   CONST4    3   3.076111  0.076111 